In [11]:
import numpy as np
import random
import pymorphy3
import numpy as np
import math
import pickle
import dill
from razdel import tokenize

from dataset_builder import calculate_word_features_for_tokens, PAD_TOKEN,get_word_features
from inference import torch_model_runner, onnx_model_runner, infer
from storage import Storage

prefix = "results_big_model_GOOD_2022-05-15"
onnx_model = onnx_model_runner(f"{prefix}/model.onnx")
# with open(f"{prefix}/storage_path.dill", "rb") as f:
#     storage_path = dill.load(f)
# storage = Storage(storage_path, enable_watcher=False)
# params = storage.get_meta('params')
with open(f"params2.dill", "rb") as f:
    params = dill.load(f)
print(params)


class jsinfer:
    async def infer(arr):
        class wrapper:
            def to_py():
                return onnx_model(arr)
        return wrapper

from stream import Stream
import functools
from collections import deque
import random
random.seed(42)

@functools.lru_cache(maxsize=128)
def get_word_features_cached(word):
    return get_word_features(word, params).numpy()



{'VARIANTS_CNT': 3, 'TARGET_CLASSES_COUNT': 3, 'INPUT_WORDS_CNT': 32, 'PUNCTUATION_TARGET': {'$empty': 0, ',': 1, '.': 2, '!': 2, '?': 2}, 'USE_NAVEC': True, 'CUT_NAVEC_TAGS_SET': set(), 'RETAIN_LEFT_PUNCT': True, 'type': 'lenta', 'NON_PUNCT_PROB': 0.5, 'INFECTED_TEXT_PROB': 0.5, 'INFECT_TYPE_PROBS': {'REPLACE_UNDEF': 0.1, 'INCORRECT_PUNCT_RANDOM_PLACE': 0.1, 'INCORRECT_PUNCT_CENTER': 0.2, 'CORRECT_PUNCT_CENTER': 0.3, 'CORRECT_PUNCT_RIGHT': 0.3}, 'feature_tags_dict': {'1per': 0, '2per': 1, '3per': 2, 'ADJF': 3, 'ADJS': 4, 'ADVB': 5, 'Abbr': 6, 'Apro': 7, 'CAPITALIZED': 8, 'COMP': 9, 'CONJ': 10, 'Fixd': 11, 'GRND': 12, 'INFN': 13, 'INTJ': 14, 'LATIN': 15, 'NOUN': 16, 'NPRO': 17, 'NUMB': 18, 'NUMR': 19, 'PRCL': 20, 'PRED': 21, 'PREP': 22, 'PRTF': 23, 'PRTS': 24, 'PUNCT_COMMA': 25, 'PUNCT_DASH': 26, 'PUNCT_DOT': 27, 'PUNCT_LEFT_PARENTHESIS': 28, 'PUNCT_QUOTE': 29, 'PUNCT_RIGHT_PARENTHESIS': 30, 'Prnt': 31, 'UNKN': 32, 'VERB': 33, 'ablt': 34, 'acc2': 35, 'accs': 36, 'actv': 37, 'datv': 38,

In [40]:

class Substr:
    def __init__(self, text):
        self.text = text
    def __repr__(self) -> str:
        return f"Substring(-1, -1, {self.text})"

def d_as_str(d):
  return "<" + " ".join(map(lambda text: text.text, d))+ ">"

async def infer_optimal(params, text):
  # print("INFERCENC IS WIERD\n" * 10)
  res = []
  last_inserted_pos = 0
  def sink(token, log=False):
    nonlocal last_inserted_pos
    if token.text == "PAD": return
    if log: print('sink', token)
    if isinstance(token, Substr):
      res.append(token.text)
      if log: print("added1 ", f"`{token.text}`", token)
    else:
      if last_inserted_pos != token.start:
        res.append(text[last_inserted_pos: token.start])
        if log: print("added2 ", f"`{text[last_inserted_pos: token.start]}`", last_inserted_pos, token.start)
      last_inserted_pos = token.stop
      res.append(token.text)
      if log: print("added3 ", f"`{token.text}`", token)

  def skip(token, log=False):
    nonlocal last_inserted_pos
    last_inserted_pos = token.stop
    if log: print('skip', token)

  def sink_remaining():
     res.append(text[last_inserted_pos:])


  async def predict_on_tokens(window_left, window_right, return_probas):
    features = [get_word_features_cached(i.text) for i in Stream(window_left).chain(window_right)]
    features_for_batch = np.stack((features, ))
    arr = np.ascontiguousarray(features_for_batch, dtype=np.float32)
    output_probas = np.array((await jsinfer.infer(arr)).to_py())
    # output_probas[0][0] += 2.
    if return_probas:
      return params["ID_TO_PUNCTUATION"], output_probas
    punct_idx = np.argmax(output_probas).item()
    punct = params["ID_TO_PUNCTUATION"][punct_idx]
    return punct


  window_left = deque()
  window_right = deque()
  log = False
  skip_next = False
  for i in Stream.repeat(Substr(PAD_TOKEN), params['INPUT_WORDS_CNT_LEFT']) \
      .chain(Stream(tokenize(text))) \
      .chain(Stream.repeat(Substr(PAD_TOKEN), params["INPUT_WORDS_CNT_RIGHT"])):
    window_right.append(i)
    if len(window_right) <= params["INPUT_WORDS_CNT_RIGHT"]:
        continue
    assert len(window_right) == params["INPUT_WORDS_CNT_RIGHT"] + 1

    next_ = window_right.popleft()
    sink(next_)
    window_left.append(next_)
    if len(window_left) < params['INPUT_WORDS_CNT_LEFT']:
      continue

    assert len(window_left) == params["INPUT_WORDS_CNT_LEFT"]
    assert len(window_right) == params["INPUT_WORDS_CNT_RIGHT"]

    if skip_next or window_right[0].text in '?!':
      prediction = "$skip"
    else:
      # params["ID_TO_PUNCTUATION"], output_probas
      prediction = await predict_on_tokens(window_left, window_right, return_probas=False)


    #random.choice([" ", "."])
    if log: print(d_as_str(window_left).rjust(100), prediction.center(6), d_as_str(window_right))

    def is_replaceable_punct(punct):
      return punct in ',.'

    if prediction == "$skip":
      pass
    elif prediction != "$empty":
      if is_replaceable_punct(window_right[0].text):
        if window_right[0].text != prediction:
          window_right[0].text = prediction
      else:
        window_left.append(Substr(prediction))
        sink(window_left[-1])
    else:
      if is_replaceable_punct(window_right[0].text):
          skip(window_right.popleft())

    skip_next = is_replaceable_punct(window_right[0].text) or window_right[0].text in '?!'

    while len(window_left) != params['INPUT_WORDS_CNT_LEFT'] - 1:
      token = window_left.popleft()

    if log: print(d_as_str(window_left).rjust(100), "      ", d_as_str(window_right))

  for i in window_right:
    sink(i)
  sink_remaining()
  ress = "".join(res)
  return ress

await infer_optimal(params, "я кое-как резолвлю но не уверен что итоговый результат заработае??")

'я кое-как резолвлю, но не уверен, что итоговый результат заработае??'